For the [Kaggel Expedia Hotel Recommendations](https://www.kaggle.com/c/expedia-hotel-recommendations/data) I have an SFrame of hotel search log data.

In [28]:
import graphlab as gl

sf = gl.SFrame({'srch_destination_id':[1, 1, 1, 1, 2, 2, 2], 
                'relevance':[4.95, 3.9, 0.05, 1.1, 0.2, 1.3, 0.05], 
                'hotel':[30, 63, 60, 25, 48, 82, 58]})

For each srch_destination_id I want to return the top 5 hotels, by relevance. Using a panda I could write something like this:

    groupby('srch_destination_id').apply(lambda group, topk(group, 'relevance' k=5) 

GraphLab doesn't have groupby apply nor a an aggrigation that returns the top 5. This made me think:

1. It would be nice to have ARGMAX/ARGMIN extended to return the k MAX/Min items as a list
2. I can see a lot of value in a groupby().apply() function. In many ways its the same way groupby works with timeseries.

To implement this in GraphLab I wrote the code below. Is there a better way to do this in GraphLab. Also note there is an odd behaviour in apply that turns ints() into floats()

In [29]:
# For each srch_destination_id create a dictonary of relevance:hotel usign CONCAT
summary = sf.groupby('srch_destination_id',{'hotel_cluster': gl.aggregate.CONCAT('relevance', 'hotel')})
summary

srch_destination_id,hotel_cluster
2,"{1.3: 82, 0.2: 48, 0.05:58} ..."
1,"{4.95: 30, 3.9: 63, 1.1:25, 0.05: 60} ..."


In [30]:
# Create a function to return the top k items in the diction by relevance
def arg_max(d, k=3):
    topk = sorted(d.keys(), reverse=True)[:k]
    return [d[k] for k in topk]

arg_max({1.3: 82, 0.2: 48, 0.05:58})

[82, 48, 58]

In [31]:
# Apply this function to each row to create the top 5 resules by releance, with the most relevant first.
# NOTE - for some reason this change the hotel id from an int() to a float()
summary['hotel_cluster'] = summary['hotel_cluster'].apply(lambda d: arg_max(d, k=5))
summary

srch_destination_id,hotel_cluster
2,"[82.0, 48.0, 58.0]"
1,"[30.0, 63.0, 25.0, 60.0]"
